#**Module 4: Data Wrangling Part 2**
In this module, you will learn how to
* Transform data
* De-duplicate data
* Substitute data
* Discretize data

**Be sure to expand all the hidden cells, run all the code, and do all the exercises--you will need the techniques for the lesson lab!**

#**What is Data Transformation?**

Watch the video below.

In [1]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/kx3sOqW5zj4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Data Transformation is just like this: We put one type of data into our "transformation" hat, like strings or categorical data, do our magic, and out comes something else--often numbers, buckets, or other categories. We don't change the size of the hat while we transform the data, that is, we don't change the internal relationships between the data, but we do change their format or the way they are organized. Most often, we change numeric data to different scales, usually between 0 and 1, which is called "Normalization.'

So, when we wrangle data, we transform and map data from one "raw" data format (like string, categorical, or numeric) into another format in order to make it usable for a number of analytical purposes. Other terms used for Data Wrangling are Preprocessing or Data Munging. Whatever the word, the goal is to bend the data to our formatting and subsetting goals so that we can use them with the algorithms and other math that we want to use.

Let's get started!

#**0. Preparation and Setup**
We are working with our adult dataset again, so we're loading our libraries and our dataset just like last time.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Reading in the data as adult dataframe
adult = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/adult.data.simplified25.csv")

#Verifying that we can see the data
adult.head()

And here is the adult_small dataframe that you built in the previous notebook. We will use it for a small, fast, proof-of-concept before we use more resources for the larger computation.

In [ ]:
adult_small=adult.iloc[70:80,]
adult_small

# **1. Data Transformation**
As you have seen, numeric datatypes allow you to do much more fun math than string datatypes. You can count them, sum them, average them, boxplot them ... all that, while string datatypes, well they can be counted, so much is true--but that really is it.


We want to transform a categorical attribute, in this case 'race', into a numeric datatype. The Python format for this operation is `DataFrame.astype(dtype, copy=True, errors='raise')`
In our case, this would be something like
* `adult_small.race.astype('int32')`
* `adult_small.race.astype('category')`
* `adult_small.race.astype('category', ordered=True)`

Let's get started. First, let's verify our datatype for 'race'



In [ ]:
adult_small.race.dtypes

'O' means 'object', which is basically a string value. Let's now transform our 'race' attribute to numeric values. Below are all the steps.

## **1.1 Adding a column with the data we want to transform**
We could theoretically transform the 'race' attribute in place, but if our code causes problems, we could mess up our entire dataframe. So, instead, we will first add a new column into which we will copy the contents of the 'race' attribute. In a second step, we will transform the values in that new column.

Detailed explanations are in the code comments below.

In [ ]:
# Creating a new attribute and populating it with the contents of the attribute that we want to transform.
race_num=adult_small.race

# Adding the new attribute to the dataframe
adult_small['race_num'] = race_num

# Checking the contents of the new attribute
adult_small.race_num

In [ ]:
# Let's check the shape now. Instead of 12 attributes, we should have 10 rows and 13 columns now because we have built a new race_num attribute
adult_small.shape

In [ ]:
# Now that we have the race_num column built, let's check the datatypes of both attributes.
adult_small.dtypes[['race','race_num']]

## **1.2 Transforming the values**
To transform an object value to numeric, we need to get the numbers from somewhere. In our case, we will transform the object to category. That gives us the index numbers for the category. Then we will replace the string values with these index numbers for the category--and voila! we have a numeric transformation.
(If you see any attribute replacement warnings as you run the code below, you can ignore them.)

In [ ]:
# Here, we convert the race_num values to categorical (to obtain the index numbers)
adult_small['race_num']= adult_small['race_num'].astype('category')

# Now we replace the categories with their index numbers, i.e. the category codes
adult_small['race_num']= adult_small['race_num'].cat.codes

# Let's see what the datatypes look like now
adult_small.dtypes[['race','race_num']]

In [ ]:
# Now let's display the contents of both attributes to double-check
adult_small[['race','race_num']]

**SUCCESS!** Now that we have worked the code out for our small proof-of-concept dataframe, we can apply it to the entire adult dataframe.

In [ ]:
# First, we back the dataframe up.
adult2=adult

# Now we build the column and copy the original values into it
adult2['race_num'] = adult2.race

# Third, we convert race_num into categorical
adult2['race_num']= adult2['race_num'].astype('category')

# Then, we replace the strings with the category indices
adult2['race_num']= adult2['race_num'].cat.codes

# Now, we check the data types
adult2.dtypes[['race','race_num']]

**YAY!** That worked, too. But how did pandas assign the numbers? Let's find out:

In [ ]:
# Making a temporary new dataframe because it's so much easier to work with just 2 attributes
race_exploration=adult2[['race','race_num']]
race2=race_exploration.drop_duplicates()
race2.sort_values(by=['race_num'])

##**Your Turn**
Explain in a couple of complete sentences the logic with which pandas haa ssigned the numbers for the race_num array.

# **2. De-Duplication**
Check out this line in the example above:
`race2=race_exploration.drop_duplicates()`

This is literally all there is to de-duplication.
For more on drop_duplicates() see, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html

# **3. Substituting Values in Dataframes**

OK--tiger, meet lion, and lion--meet rhino!

<img src="https://github.com/shstreuber/Data-Mining/blob/master/images/transformation.jpg?raw=true">

Here is how to substitute values in a dataframe. For this purpose, we will try out the df.replace method shown in https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html. This really convenient method works for strings and numeric values alike.

Here, we will work with the workclass attribute.

In [ ]:
adult3 = adult
adult3.workclass.unique()

### The Problem
Oh, look: We have a '?' value in there. This could be any of the following
* Misformatted data that might have accidentally broken during the E of [ETL](https://www.sas.com/en_us/insights/data-management/what-is-etl.html).
* Some sort of placeholder character that our data warehouse has inserted for missing values.
* Intentional and supposed to stand for 'Unknown'.

Whatever it is, we may never know. That is one of the big problems when dealing with big datasets. The next problem is:
**bold text**
> **What are we going to do with it?**

Are we going to just **ignore** the rows in which the '?' appears--or are we going to **do something about it**? To answer that question, it is helpful to know how many rows contain the '?'



In [ ]:
adult3[adult3['workclass'] == '?']

Hm. That's 1,399 rows out of 25,000--5.5% of all the data. At this point, it's a good idea to ask your supervisor, that is, the client of your analysis, what they would prefer. Let's say the client comes back to say they **don't** want you to drop 6% of the dataset.

At that point, you will have to make an assumption--and that's where critical thinking skills come in: Does it matter whether there's an accidental or an intentional '?'. If it does, then further research is needed. If it does NOT, then it might be safe to assume that the '?' stands for 'Unknown'.

That's why we will translate '?' to 'Unknown'.

In [ ]:
# Instead of building another column to see any transformations, we will substitute data in place this time around.
# For this purpose, we're using the replace() method from pandas
adult3.workclass.replace('?','Unknown', inplace=True)

# Did we indeed replace all '?'? Let's check:
adult3[adult3['workclass'] == '?']

In [ ]:
# Looks like it worked. Let's check if the new value appears in the list of unique values.
adult3.workclass.unique()

## **Your Turn**
Take a look at the unique values in the occupation attribute. Do you see something similar as what we solved above?
1. Replace the unusual value with 'NA'
2. Then count how many rows contain 'NA' values

# **4. Discretization/ Aggregation**
Remember how I said we're also going to exchange the lion and the tiger from the Rocky and Bullwinkle cartoon into a "Big Cats" category? That's essentially what Discretization is about.

<img src = "https://github.com/shstreuber/Data-Mining/blob/master/images/aggregation.jpg?raw=true">

Data discretization is typically defined as a process of converting continuous data attribute values into a finite set of intervals and associating with each interval some specific data value. So, you could say that the goal of discretization is to simplify data by putting them into summary categories. While discretization is typically applied to numeric data only, we can also use this concept to simplify other data (like object-type data or data that has too many categories to be useful).


##**4.1 Data Summarization**
We're not done with the workclass attribute just yet. As you have seen, we have 9 categories. We will reduce these to the following:
- Unknown--contains all Unknown, Never-worked, and Without-pay
- Government--contains all Federal, State, and Local government employees
- Self--contains all self-employed groups

This summarization will allow us to more easily group our data by the three larger units in the workclass attribute.

In [ ]:
# Let's check the categories in the workclass attribute!
adult3.workclass.unique()

Given the structure of this attribute, we are seeing three "natural" groups: Unknown, Government, and self-employed people. This means we can simplify the data we have and create three big aggregations with simple substitutions:

In [16]:
#Summarizing all the Unknowns
adult3.workclass.replace('Unknown','Unknown', inplace=True)
adult3.workclass.replace('Never-worked','Unknown', inplace=True)
adult3.workclass.replace('Without-pay','Unknown', inplace=True)

In [18]:
#Summarizing all the Government attributes
adult3.workclass.replace('State-gov', 'Government', inplace=True)
adult3.workclass.replace('Federal-gov', 'Government', inplace=True)
adult3.workclass.replace('Local-gov', 'Government', inplace=True)

##**Your Turn**
Complete the job! Summarize the other three groups into the "Self" category.

##**4.2 Binning and Bucketing (Numeric Discretization)**
We have arrived at the "pure" meaning of discretization--the one you find in most textbooks. In this context, you'll often hear the terms binning and bucketing.

In short, Data binning and bucketing groups data in bins or buckets, in the sense that it replaces values contained into a small interval with a single representative value for that interval. It includes also dealing with missing values, formatting, normalization and standardization. That makes it a technique for data smoothing.

The **difference** between binning and bucketing is:
* In **bucketing**, we define the boundaries for the categories ourselves. They don't have to be the same size.
* In **binning**, we define the number of categories, and the code distributes them evenly across our dataset, depending on whether we set the binning criterion to the number of data in each bin, or to the value of the data.

On a technical level, we to convert numeric values to categorical or to sample (quantise) numeric values, sometimes using distance (remember distance from one of the previous modules?) and sometimes using frequency. we can also reduce numeric values through sampling.

Normalization and standardization are special cases of Discretization that we'll talk about in a bit. To learn more in depth about binning, go [here](https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950). To find out more about how to do this at a practical level, take a look at [this explanation](https://stackoverflow.com/questions/45273731/binning-column-with-python-pandas).

###**Binning**
It's time to look at incomeUSD. To remind yourself what the attribute looks like, use the describe() function to produce a 5-number summary:

In [19]:
adult.incomeUSD.describe()

count     25000.000000
mean      56382.802560
std       44128.661114
min       20002.000000
25%       29963.750000
50%       39689.000000
75%       49588.500000
max      199960.000000
Name: incomeUSD, dtype: float64

We see that the income ranges from 20,002 to 199,960. How about making three categories: Low, medium, and high? If we are building 3 groups, we need 4 edges of intervals (**BINS**):


<img src ="https://github.com/shstreuber/Data-Mining/blob/master/images/3bins.jpg?raw=true">

* small — (edge1, edge2)
* medium — (edge2, edge3)
* high — (edge3, edge4)

We can use the linspace() function of the numpy package to calculate the 4 bins, equally distributed.

In [ ]:
bins = np.linspace(adult.incomeUSD.min(),adult.incomeUSD.max(),4)
bins

Pandas has calculated the following cutoffs:
* Group 1: 20,002.00000 to 79,988
* Group 2: 79,988 to 139,974
* Group 3: 139,974 to 199,960

Now we define the three labels (small, medium, high) for these three groups:

In [ ]:
labels = ['small', 'medium', 'high']

We can use the cut() function to convert the numeric values of incomeUSD into the categorical values of small, medium, and high. We need to specify the bins and the labels. In addition, we set the parameter include_lowest to True in order to include also the minimum value. In order not to change our original dataset, we'll create an adult3 backup dataset:

In [ ]:
adult3=adult
adult3['bins'] = pd.cut(adult3['incomeUSD'], bins=bins, labels=labels, include_lowest=True)

This gives us the new 'bins' attribute at the end of our adult3 dataframe:

In [ ]:
adult3.head()

We can now use groupby() on this new 'bins' attribute to count the data in each of these categories:

In [ ]:
adult3.groupby(['bins']).count()

We can even plot them in a histogram:

In [ ]:
plt.hist(adult3['bins'], bins=3)

Let's put our **CRITICAL THINKING** hats on: The cutoffs for the bins that we have calculated were:
* Group 1: 20,002.00000 to 79,988
* Group 2: 79,988 to 139,974
* Group 3: 139,974 to 199,960

What we are seeing in the histogram is that a huge majority of our datapoints falls into the "small" category, so below 79,988, whereas a much smaller number falls into the medium and high categories.

Are these bins useful? Not really. The only thing they show us is that we have a whole lot of poor people in our dataset. Let's go back to the 5-number summary:

In [ ]:
adult3.incomeUSD.describe()

###**Bucketing**
We see that the income ranges from 20,002 to 199,960, with a median at $39,689. That's not even a fifth of the top salary! Such a low median means that 50% of the people in our dataset make less than 39,732 while the remaining 50% inhabit the range from 39,689 to 199,960. In fact, only 25% of people earn between 49,588 (Q3) and the top salary. Wouldn't it be interesting to see how many earners we have in the top range, between, say 150,000 and 199,000?

That's where **BUCKETS** come in.

With buckets, you set the interval cutoffs manually. So, how about we make 5 buckets:
* poor (20002 to the median of 39689)
* low-wage-earners (median of 39689 to Q3 aka 49588)
* middle-class (49588 to 79988 (which was the previous Group 2 cutoff))
* rich (79988 to 150000)
* super-rich (150000 to 199960)

<img src = "https://github.com/shstreuber/Data-Mining/blob/master/images/5buckets_named_diff.jpg?raw=true">

You're probably noticing that these buckets aren't the same size. With bucketing, that's not the point. We get to make the cutoffs.

Now, let's code this.


In [ ]:
# Setting up our 5 buckets and their labels
buckets = [ 20002, 39689, 49588, 79988, 150000, 199960 ]
bucketlabels = ['poor', 'low-wage-earners', 'middle-class', 'rich', 'super-rich']

# Using cut() to separate data into the buckets we have built
adult3['buckets'] = pd.cut(adult3['incomeUSD'] , bins=buckets, labels=bucketlabels, include_lowest=True)

# Check if we have the new attribute 'buckets' at the end of our dataset
adult3.head()

In [ ]:
# Checking the contents of our buckets
adult3.groupby(['buckets']).count()

In [ ]:
# Plotting the distribution now
plt.hist(adult3['buckets'], bins=3)
plt.show()

###**CRITICAL THINKING MOMENT**
As you can see, bucketing gives us much better information about our dataset, but we needed to use binning to point us in the right direction! Now, look at these three bucket cutoffs again:

* poor (20002 to the median of 39689)
* rich (79988 to 150000)
* super-rich (150000 to 199960)

We are seeing that the number of poor people is still vastly bigger than the numbers of rich and super-rich people. **AND** we see that most of the poverty is concentrated between 20002 and 39689--that's a range of less than 20000--whereas the range of rich to super-rich income is much larger, from 79988 to 199960--that's roughly 120000!
<center>
<img src = "https://thumbs.dreamstime.com/z/mind-blown-emoji-exploding-head-emoticon-white-background-d-rendering-219279605.jpg?w=300">
</center>

**DISCLAIMER**: Of course, we are evaluating the income from an American perspective, even though the numbers come from countries all over the world, and we are not looking at buying power by country, at all. For example, $39,689 buys much more in a country like Poland than it does in the US. So, to get more balanced insights, we would have to do much more investigating!

##**4.3 Normalization and Standardization**
A third type of Discretization, which will become critical when we talk about Neural Networks, helps us compare the values inside numeric attributes in a dataset by putting them all on the same scale.

Below, I will show you the most popular method for normalization, with the MixMaxScaler from the new-to-you scikit-learn package, which transforms values to a scale from 0 to 1. Then, I will show you two methods for standardization: A basic pandas method and the StandardScaler from the--you guessed it--scikit-learn package

**CRITICAL THINKING MOMENT**
Which data would we want to compare? Typically the ones that can have meaning for one another, like the ones with the same units. So here, we will use age and educationyears to normalize because both are in the same unit: Years.

First, we will pull out these two numeric attributes into their own dataframe. That will make our processing faster and easier

In [ ]:
adult_years = adult.iloc[:,[0,3]]
adult_years.head()

In [ ]:
# Better safe than sorry! Let's check the dataypes.
adult_years.dtypes

###**Normalization with scikit-learn**
Scikit-learn is a scientific package that allows us to do biased estimating, based on how you tell the algorithm to calculate the normalization range. Some data scientists prefer the MixMax method over the pandas method because it gives more control over how to scale the attributes. The most popular approach is [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), which scales and translates each attribute individually such that it falls between zero and one.

In [ ]:
# Normalization with MinMaxScaler

from sklearn import preprocessing # the package

adult_years1 = adult_years # backing up my dataframe--you never know!
x = adult_years1.values # pulling out just the array values

# Now we are using the scaler
min_max_scaler = preprocessing.MinMaxScaler()

# Check the link above for more explanation of this line
x_scaled = min_max_scaler.fit_transform(x)

adult_years1 = pd.DataFrame(x_scaled)
adult_years1 # We should now be seeing all values between 0 and 1.

###**Standardization with scikit-learn**
Standardization is typically when values are organized around the mean and divided by the standard deviation. The fastest approach here is the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from scikit-learn.

In [ ]:
# Standardization with StandardScaler

from sklearn.preprocessing import StandardScaler # the package

adult_years2 = adult_years # backing up my dataframe--you never know!
scaler = StandardScaler() # setting an abbreviation to make things easier

#adult_years2 = scaler.fit_transform(adult_years2.to_numpy()) # if you want to add an extra transformation to a numpy array
adult_years2 = scaler.fit_transform(adult_years2)

adult_years2

### **Standardization with pandas**
Using pandas for normalization is pretty straightforward: From each data value in an attribute, we subtract the mean and divide by the standard deviation. We do this row by row, so we specify axis=0. (axis=1 means column, remember?).

Cycling through our data will require the lambda transformation we have seen in a previous module. To learn more about this lambda transformation, check [this link](https://www.analyticsvidhya.com/blog/2020/03/what-are-lambda-functions-in-python/)

In [ ]:
adult_years3 = adult_years.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
adult_years3.head()

**OH! WAIT! WHAT!!!!???**

Compare these values to the output for the StandardScaler--do you notice anything?


### **The Big Picture**
If you don't want to build a separate dataframe, that's ok, but it takes up more processing power. You will have to work with iloc to tell Python which columns to transform (I am using adult3 below with the pandas transformation just because we did that last and because I don't want to break the original adult dataframe here)

In [ ]:
adult3.iloc[:,[0,3]] = adult.iloc[:,[0,3]].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
adult3.head()

##**Your Turn**
Now apply the MinMaxScaler to the age and educationyears attribute throughout the ENTIRE adult dataset (back it up as adult4 so you can always quickly replace it). For this, you will combine code from the MinMaxScaler normalization with the code directly above that applies the transformation to specific columns/ attributes/ dimensions/ features in the dataset.